In [ ]:
import pandas as pd
import folium
from folium.plugins import BeautifyIcon, MarkerCluster, HeatMap
import branca

# Load the data from the CSV file
try:
    data = pd.read_csv('simulated-heat-stress-data.csv')
except FileNotFoundError:
    print("Error: CSV file not found. Please check the file path.")
    raise

# Verify required columns exist in the dataset
required_columns = ['latitude', 'longitude', 'heat_stress_score', 'state_province']
if not all(col in data.columns for col in required_columns):
    print(f"Error: Missing required columns. Required columns: {required_columns}")
    raise ValueError("Dataset is missing required columns")

# Create a base map centered on the USA
m = folium.Map(location=[37.0902, -95.7129], zoom_start=4)


# Define a color scale
def get_color(score):
    """Assign color based on heat stress score."""
    if score < 50:
        return 'green'
    elif score < 65:
        return 'yellow'
    elif score < 80:
        return 'orange'
    return 'red'

# Create markers with custom shapes based on heat stress level
for _, row in data.iterrows():
    # Assign shape and color based on heat stress score
    if row['heat_stress_score'] < 50:
        icon_shape = 'star'
        icon_color = 'green'
    elif row['heat_stress_score'] < 65:
        icon_shape = 'circle'
        icon_color = 'yellow'
    elif row['heat_stress_score'] < 80:
        icon_shape = 'square'
        icon_color = 'orange'
    else:
        icon_shape = 'triangle'
        icon_color = 'red'

    # Use DivIcon for custom marker shapes
    if icon_shape == 'star':
        icon_html = f'<div style="width: 20px; height: 20px; background-color:{icon_color}; clip-path:polygon(50% 0%, 61% 35%, 98% 35%, 68% 57%, 79% 91%, 50% 70%, 21% 91%, 32% 57%, 2% 35%, 39% 35%);"></div>'
    elif icon_shape == 'circle':
        icon_html = f'<div style="width: 15px; height: 15px; background-color:{icon_color}; border-radius:50%;"></div>'
    elif icon_shape == 'square':
        icon_html = f'<div style="width: 10px; height: 10px; background-color:{icon_color};"></div>'
    elif icon_shape == 'triangle':
        icon_html = f'<div style="width: 0px; height: 0px; border-left: 7px solid transparent; border-right: 7px solid transparent; border-bottom: 10px solid {icon_color};"></div>'

    # Create the DivIcon and add marker
    custom_icon = folium.DivIcon(html=icon_html)

    # Add marker with custom icon and popup
    folium.Marker(
        location=[row['latitude'], row['longitude']],
        icon=custom_icon,
        popup=folium.Popup(f"""
            <strong>State:</strong> {row['state_province']}<br>
            <strong>Latitude:</strong> {row['latitude']}<br>
            <strong>Longitude:</strong> {row['longitude']}<br>
            <strong>Heat Stress Score:</strong> {row['heat_stress_score']}<br>
        """, max_width=250),
        tooltip=f"Heat Stress: {row['heat_stress_score']} - {row['state_province']}"
    ).add_to(m)

# Add a color legend with the new shapes
legend_html = '''
     <div style="position:fixed; bottom:10px; left:10px; background:#fff;
                 border-radius:5px; z-index:9999; font-size:12px;">
     <p><strong>Heat Stress Level:</strong><br>
     <i class="fa fa-star" style="color:green"></i> Low (0-50) - Star-shaped Green<br>
     <i class="fa fa-circle" style="color:yellow"></i> Moderate (50-65) - Circle Yellow<br>
     <i class="fa fa-square" style="color:orange"></i> High (65-80) - Square Orange<br>
     <i class="fa fa-circle" style="color:red"></i> Extreme (80+) - Triangle Red</p>
     </div>
'''
m.get_root().html.add_child(folium.Element(legend_html))

# Save map to HTML
m.save('custom_heat_stress_map.html')

m
